<a href="https://colab.research.google.com/github/Fordalo/Data_SCIENCE_WORK/blob/main/WOrldHappiness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First We want to load the data sets into our environment to begin cleaning and organizing

In [1]:
import pandas as pd
import numpy as np
import datetime
import statsmodels.api as sm 
from statsmodels.formula.api import ols
from matplotlib import pyplot as plt


#moutning google drive to google colab
#allows me to acces google drive files
from google.colab import drive

drive.mount('/content/gdrive')

happiness2015 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Data Science projects/World Happiness/2015.csv')
happiness2016 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Data Science projects/World Happiness/2016.csv')
happiness2017 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Data Science projects/World Happiness/2017.csv')
happiness2018 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Data Science projects/World Happiness/2018.csv')
happiness2019 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Data Science projects/World Happiness/2019.csv')
happiness2015.describe()


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Mounted at /content/gdrive


,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
count,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000
mean,79.493671,5.375734,0.047885,0.846137,0.991046,0.630259,0.428615,0.143422,0.237296,2.098977
std,45.754363,1.145010,0.017146,0.403121,0.272369,0.247078,0.150693,0.120034,0.126685,0.553550
min,1.000000,2.839000,0.018480,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.328580
25%,40.250000,4.526000,0.037268,0.545808,0.856823,0.439185,0.328330,0.061675,0.150553,1.759410
50%,79.500000,5.232500,0.043940,0.910245,1.029510,0.696705,0.435515,0.107220,0.216130,2.095415
75%,118.750000,6.243750,0.052300,1.158448,1.214405,0.811013,0.549092,0.180255,0.309883,2.462415
max,158.000000,7.587000,0.136930,1.690420,1.402230,1.025250,0.669730,0.551910,0.795880,3.602140


I want to add a column for year in each dataset and view it as a categorical attribute. I want to see if the actual year has any impact on the happiness score of the country. I also want to check the dimensions of each data fram to see if I can row stack the data

In [2]:
from pandas.core.algorithms import unique
happiness2015['YEAR'] = '2015'
happiness2016['YEAR'] = '2016'
happiness2017['YEAR'] = '2017'
happiness2018['YEAR'] = '2018'
happiness2019['YEAR'] = '2019'

#creating a list of dataframe objects
happiness_arr = [happiness2015, happiness2016, happiness2017, happiness2018, happiness2019]
for i in happiness_arr:
  print(i.columns, i.shape)

Index(['Country', 'Region', 'Happiness Rank', 'Happiness Score',
       'Standard Error', 'Economy (GDP per Capita)', 'Family',
       'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)',
       'Generosity', 'Dystopia Residual', 'YEAR'],
      dtype='object') (158, 13)
Index(['Country', 'Region', 'Happiness Rank', 'Happiness Score',
       'Lower Confidence Interval', 'Upper Confidence Interval',
       'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)',
       'Freedom', 'Trust (Government Corruption)', 'Generosity',
       'Dystopia Residual', 'YEAR'],
      dtype='object') (157, 14)
Index(['Country', 'Happiness.Rank', 'Happiness.Score', 'Whisker.high',
       'Whisker.low', 'Economy..GDP.per.Capita.', 'Family',
       'Health..Life.Expectancy.', 'Freedom', 'Generosity',
       'Trust..Government.Corruption.', 'Dystopia.Residual', 'YEAR'],
      dtype='object') (155, 13)
Index(['Overall rank', 'Country or region', 'Score', 'GDP per capita',
       'S

As we can see, all of these dataframes have different dimensions. Its also worth noting that all of the datasets do not have the same amount of N. My next few steps to perform my OLS will be the following:



1.   Skim through the columns of the dataset, and select common columns amongst all
2.   Merge all of the datasframes via rowstack
3.   Remove columns we dont need

In [4]:
happiness2015[['Happiness Score', "Country", 'Economy (GDP per Capita)', 'Health..Life.Expectancy.', 'Freedom', 'Trust (Government Corruption)', 'YEAR']]

,Happiness Score,Country,Economy (GDP per Capita),Health (Life Expectancy),Freedom,Trust (Government Corruption),YEAR
0,7.587,Switzerland,1.39651,0.94143,0.66557,0.41978,2015
1,7.561,Iceland,1.30232,0.94784,0.62877,0.14145,2015
2,7.527,Denmark,1.32548,0.87464,0.64938,0.48357,2015
3,7.522,Norway,1.45900,0.88521,0.66973,0.36503,2015
4,7.427,Canada,1.32629,0.90563,0.63297,0.32957,2015
...,...,...,...,...,...,...,...
153,3.465,Rwanda,0.22208,0.42864,0.59201,0.55191,2015
154,3.340,Benin,0.28665,0.31910,0.48450,0.08010,2015
155,3.006,Syria,0.66320,0.72193,0.15684,0.18906,2015
156,2.905,Burundi,0.01530,0.22396,0.11850,0.10062,2015


In [21]:
happiness2015 = happiness2015[['Happiness Score', "Country", 'Economy (GDP per Capita)', 'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)', 'YEAR']]
happiness2016 = happiness2016[['Happiness Score', 'Country', 'Economy (GDP per Capita)', 'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)','YEAR']]
happiness2017 = happiness2017[['Happiness.Score', "Country", 'Economy..GDP.per.Capita.', 'Health..Life.Expectancy.', 'Freedom', 'Trust..Government.Corruption.', 'YEAR']]
happiness2018 = happiness2018[['Score', 'Country or region', 'GDP per capita', 'Healthy life expectancy', 'Freedom to make life choices', 'Perceptions of corruption', 'YEAR']]
happiness2019 = happiness2019[['Score', 'Country or region', 'GDP per capita', 'Healthy life expectancy', 'Freedom to make life choices', 'Perceptions of corruption', 'YEAR']]


Now that we have created our happiness datasets for each year, We have a couple of things to think about. Here are the list of data types for each column

1. Score = continuous
2. Country = **Categorical**
3. GDP = Continuous
4. Health = Continuous
5. Freedom = Continuous
6. Corruption = COntinuous
7. YEAR = Categorical, but it can be casted as a date object which can be viewed as continous

We also need to understand that all of our datasets have different size 'N', therefore, there are some countries that are not in each dataset. Due to this fact, I would first like to perform regular OLS on each dataset. Before we do this, we need to dumbify our categorical column 'Country'.
